In [15]:
import pandas as pd
import numpy as np
import glob, os

# Get CSV files list from a folder
path = 'data/crashes_data'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file, encoding = 'latin1') for file in csv_files)


#encoding moest hier gedefinieerd worden anders kreeg ik een error, https://stackoverflow.com/questions/50342517/unicodedecodeerror-reading-a-csv-file

# Concatenate all DataFrames
df   = pd.concat(df_list, ignore_index=True)

In [17]:
df = df[["Date","Package Name","Daily Crashes","Daily ANRs"]]

KeyError: "None of [Index(['Date', 'Package Name', 'Daily Crashes', 'Daily ANRs'], dtype='object')] are in the [columns]"